# Start van het project

1. Data uit mongoDB halen om in een csv te steken.

In [4]:
# extract query result to
%matplotlib inline
%reset
low_memory=False
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
import math
import os


client = MongoClient("mongodb://localhost:27017/")
 
db = client["lifesnaps"]
collection = db["fitbit"]
doc = collection.find_one({"type": "estimated_oxygen_variation"})
print(doc)

{'_id': ObjectId('62cc1f9ab41dcd4b1beae820'), 'id': ObjectId('621e2e8e67b776a24055b564'), 'type': 'estimated_oxygen_variation', 'data': {'timestamp': '05/24/21 01:03:30', 'Infrared to Red Signal Ratio': -4}}


In [ ]:

# test query
# from bson import ObjectId

# query = {
#     "type": "lightly_active_minutes"
# }
# docs = collection.find(query).limit(10)
# for doc in docs:
#     print(doc)

# Adjust your query as needed



# query = {
#     "type": "sleep"
# }  
# dataSleep = list(collection.find(query))

# query = {
#     "type": "steps"
# }
# dataSteps = list(collection.find(query))

# query = {
#     "type": "lightly_active_minutes"
# }
# dataLightlyActiveMinutes = list(collection.find(query))

# query = {
#     "type": "moderately_active_minutes"
# }
# dataModeratlyActiveMinutes = list(collection.find(query))

# query = {
#     "type": "very_active_minutes"
# }
# dataVeryActiveMinutes = list(collection.find(query))


# Kobe

query = {
    "type": "sedentary_minutes"
}
dataSedentaryMinutes = list(collection.find(query))

query = {
    "type": "Profile"
}
dataProfile = list(collection.find(query))

query = {
    "type": "Daily Heart Rate Variability Summary"
}
dataDailyHeartRateVariabilitySummary = list(collection.find(query)) 

query = {
    "type": "resting_heart_rate"
}
dataRestingHeartRate = list(collection.find(query))

KeyboardInterrupt: 

In [9]:
query = {
    "type": "very_active_minutes"
}
dataVeryActiveMinutes = list(collection.find(query))

df = pd.DataFrame(dataVeryActiveMinutes)

csv = "very_active_minutes"

df.to_csv(f"{csv}.csv", index=False)

print(f"Data has been saved to {csv}.csv")


Data has been saved to very_active_minutes.csv
